Santander Customer Transaction Prediction
We need 0.9 score on roc_auc_score
So let's use a couple of potentially useful models - LR with/without PCA, lightgbm and a Linear Sequential Neural Network

In [2]:
from sklearn.decomposition import PCA

pca = PCA()

In [3]:
import pandas as pd
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
train_df.head(5)

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,...,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,2.9252,3.1821,14.0137,0.5745,8.7989,14.5691,5.7487,-7.2393,4.2840,30.7133,10.5350,16.2191,2.5791,2.4716,14.3831,13.4325,-5.1488,-0.4073,4.9306,5.9965,-0.3085,12.9041,-3.8766,16.8911,11.1920,10.5785,0.6764,7.8871,...,15.4576,5.3133,3.6159,5.0384,6.6760,12.6644,2.7004,-0.6975,9.5981,5.4879,-4.7645,-8.4254,20.8773,3.1531,18.5618,7.7423,-10.1245,13.7241,-3.5189,1.7202,-8.4051,9.0164,3.0657,14.3691,25.8398,5.8764,11.8411,-19.7159,17.5743,0.5857,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,-0.4032,8.0585,14.0239,8.4135,5.4345,13.7003,13.8275,-15.5849,7.8000,28.5708,3.4287,2.7407,8.5524,3.3716,6.9779,13.8910,-11.7684,-2.5586,5.0464,0.5481,-9.2987,7.8755,1.2859,19.3710,11.3702,0.7399,2.7995,5.8434,...,29.4846,5.8683,3.8208,15.8348,-5.0121,15.1345,3.2003,9.3192,3.8821,5.7999,5.5378,5.0988,22.0330,5.5134,30.2645,10.4968,-7.2352,16.5721,-7.3477,11.0752,-5.5937,9.4878,-14.9100,9.4245,22.5441,-4.8622,7.6543,-15.9319,13.3175,-0.3566,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,-0.3249,-11.2648,14.1929,7.3124,7.5244,14.6472,7.6782,-1.7395,4.7011,20.4775,17.7559,18.1377,1.2145,3.5137,5.6777,13.2177,-7.9940,-2.9029,5.8463,6.1439,-11.1025,12.4858,-2.2871,19.0422,11.0449,4.1087,4.6974,6.9346,...,13.2070,5.8442,4.7086,5.7141,-1.0410,20.5092,3.2790,-5.5952,7.3176,5.7690,-7.0927,-3.9116,7.2569,-5.8234,25.6820,10.9202,-0.3104,8.8438,-9.7009,2.4013,-4.2935,9.3908,-13.2648,3.1545,23.0866,-5.3000,5.3745,-6.2660,10.1934,-0.8417,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,2.3061,2.8102,13.8463,11.9704,6.4569,14.8372,10.7430,-0.4299,15.9426,13.7257,20.3010,12.5579,6.8202,2.7229,12.1354,13.7367,0.8135,-0.9059,5.9070,2.8407,-15.2398,10.4407,-2.5731,6.1796,10.6093,-5.9158,8.1723,2.8521,...,31.8833,5.9684,7.2084,3.8899,-11.0882,17.2502,2.5881,-2.7018,0.5641,5.3430,-7.1541,-6.1920,18.2366,11.7134,14.7483,8.1013,11.8771,13.9552,-10.4701,5.6961,-3.7546,8.4117,1.8986,7.2601,-0.4639,-0.0498,7.9336,-12.8279,12.4124,1.8489,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,-9.4458,-12.1419,13.8481,7.8895,7.7894,15.0553,8.4871,-3.0680,6.5263,11.3152,21.4246,18.9608,10.1102,2.7142,14.2080,13.5433,3.1736,-3.3423,5.9015,7.9352,-3.1582,9.4668,-0.0083,19.3239,12.4057,0.6329,2.7922,5.8184,...,33.5107,5.6953,5.4663,18.2201,6.5769,21.2607,3.2304,-1.7759,3.1283,5.5518,1.4493,-2.6627,19.8056,2.3705,18.4685,16.3309,-3.3456,13.5261,1.7189,5.1743,-7.6938,9.7685,4.8910,12.2198,11.8503,-7.8931,6.4209,5.9270,16.0201,-0.2829,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA

print(train_df.shape)

lr_clf = LogisticRegression()
y_train = np.array(train_df['target'])
print(train_df.columns)
X_train = train_df.drop('target', axis=1).drop('ID_code', axis=1)

scaler = MinMaxScaler()
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)

scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)


# pca = PCA()
# pca.fit(X_train)
# X_train_pca = pca.transform(X_train)
# X_val_pca = pca.transform(X_val)

lr_clf.fit(X_train, y_train)

print(roc_auc_score(y_val, lr_clf.predict_proba(X_val)[:, 1]))

(200000, 202)
Index(['ID_code', 'target', 'var_0', 'var_1', 'var_2', 'var_3', 'var_4',
       'var_5', 'var_6', 'var_7',
       ...
       'var_190', 'var_191', 'var_192', 'var_193', 'var_194', 'var_195',
       'var_196', 'var_197', 'var_198', 'var_199'],
      dtype='object', length=202)
0.8590830509196538


In [ ]:
from sklearn.dummy import DummyClassifier

dummy = DummyClassifier(strategy='most_frequent').fit(X_train, y_train)
print(roc_auc_score(y_val, dummy.predict_proba(X_val)[:, 1]))

0.5


In [ ]:
import lightgbm as lgb

train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

params = {
    'objective': 'binary',
    'metric': 'auc',                # Можно несколько: ['auc', 'binary_logloss']
    'boosting_type': 'gbdt',
    'num_leaves': 31,                # Увеличивай для сложных зависимостей (но осторожно с переобучением)
    'learning_rate': 0.05,           # Если данных много, можно 0.1, если мало - меньше
    'bagging_freq': 5,                # Частота бэггинга
    'verbose': -1,                    # Подавляем вывод (чтобы не спамил)
    'num_threads': -1,                 # Использовать все ядра
    'random_state': 42,
    # Параметры регуляризации (помогают от переобучения)
    'lambda_l1': 0.1,
    'lambda_l2': 0.1,
    'min_child_samples': 20,           # Минимум данных в листе
}

model = lgb.train(
    params,
    train_set=train_data,
    valid_sets=[valid_data]
)

y_pred_prob = model.predict(X_val, num_iteration=model.best_iteration)
print(y_pred_prob.shape)

(60000,)


In [ ]:
y_pred = (y_pred_prob > 0.5).astype(int)
print(roc_auc_score(y_val, y_pred))

0.508004834578461


In [7]:
import torch.nn as nn
import torch
from torch.utils.data import DataLoader, TensorDataset


input_size = 200

model = nn.Sequential(
    nn.Linear(input_size, 300),
    nn.ReLU(),
    nn.BatchNorm1d(300),
    nn.Dropout(0.2),
    nn.Linear(300, 600),
    nn.ReLU(),
    nn.BatchNorm1d(600),
    nn.Dropout(0.2),
    nn.Linear(600, 64),
    nn.ReLU(),
    nn.BatchNorm1d(64),
    nn.Dropout(0.2),
    nn.Linear(64, 2),
    nn.ReLU()
)

X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train)
X_val_tensor = torch.FloatTensor(X_val)
y_val_tensor = torch.LongTensor(y_val)

# Создание DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Инициализация модели
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model = model.to(device)


def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    return total_loss / len(loader), 100 * correct / total

def validate_with_auc(model, loader, criterion, device):
    """
    Валидация с расчетом ROC-AUC
    """
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    # Собираем все предсказания и метки
    all_probs = []  # Вероятности для positive класса
    all_labels = [] # Истинные метки

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Получаем вероятности через softmax
            probs = torch.softmax(outputs, dim=1)

            # Берем вероятность positive класса (индекс 1)
            positive_probs = probs[:, 1].cpu().numpy()

            # Сохраняем
            all_probs.extend(positive_probs)
            all_labels.extend(labels.cpu().numpy())

            # Считаем accuracy
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Конвертируем в numpy массивы
    all_probs = np.array(all_probs)
    all_labels = np.array(all_labels)

    # Считаем ROC-AUC
    try:
        auc_score = roc_auc_score(all_labels, all_probs)
    except ValueError as e:
        print(f"Ошибка при расчете AUC: {e}")
        auc_score = 0.5  # Значение по умолчанию

    return {
        'loss': total_loss / len(loader),
        'accuracy': 100 * correct / total,
        'auc': auc_score,
        'probabilities': all_probs,
        'true_labels': all_labels
    }

# Основной цикл тренировки
num_epochs = 50
best_val_acc = 0
best_auc = 0
best_epoch = 0
history = {'train_loss': [], 'val_loss': [], 'val_acc': [], 'val_auc': []}

for epoch in range(num_epochs):
    # Тренировка
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)

    # Валидация
    val_results = validate_with_auc(model, val_loader, criterion, device)

    # Сохраняем историю
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_results['loss'])
    history['val_acc'].append(val_results['accuracy'])
    history['val_auc'].append(val_results['auc'])

    # Сохраняем лучшую модель по AUC
    if val_results['auc'] > best_auc:
        best_auc = val_results['auc']
        best_epoch = epoch + 1
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'auc': best_auc,
            'accuracy': val_results['accuracy']
        }, 'best_model_auc.pth')
        print(f"✅ Эпоха {epoch+1}: Новая лучшая модель! AUC = {best_auc:.4f}")

print(f'\n🏆 Лучшая точность валидации: {best_val_acc:.2f}%')

✅ Эпоха 1: Новая лучшая модель! AUC = 0.8431
✅ Эпоха 2: Новая лучшая модель! AUC = 0.8527
✅ Эпоха 3: Новая лучшая модель! AUC = 0.8555
✅ Эпоха 6: Новая лучшая модель! AUC = 0.8573
✅ Эпоха 7: Новая лучшая модель! AUC = 0.8573


KeyboardInterrupt: 